# 1. AutoEncoder Demo (FC + different hidden layers)

### About this notebook

This notebook was used in the 50.039 Deep Learning course at the Singapore University of Technology and Design.

**Author:** Matthieu DE MARI (matthieu_demari@sutd.edu.sg)

**Version:** 1.1 (05/04/2022)

**Requirements:**
- Python 3 (tested on v3.9.6)
- Matplotlib (tested on v3.5.1)
- Numpy (tested on v1.22.1)

### Imports

In [ ]:
# Imports
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import save_image
import matplotlib.pyplot as plt

In [ ]:
# CUDA check
CUDA = True
device = "cuda" if (torch.cuda.is_available() and CUDA) else "cpu"
print(torch.cuda.is_available())
print(device)

### Dataset and dataloaders

In [ ]:
# Data Preprocessing
# - ToTensor
# - Image Normalization
transform = transforms.Compose([transforms.ToTensor(), \
                                transforms.Normalize((0.1307,), (0.3081,))])

In [ ]:
# Train datasets/dataloaders
train_set = torchvision.datasets.MNIST(root='./data', \
                                       train = True, \
                                       download = True, \
                                       transform = transform)
train_loader = torch.utils.data.DataLoader(train_set, \
                                           batch_size = 32, \
                                           shuffle = False)

In [ ]:
# Test datasets/dataloaders
test_set = torchvision.datasets.MNIST(root = './data', \
                                      train = False, \
                                      download = True, \
                                      transform = transform)
test_loader = torch.utils.data.DataLoader(test_set, \
                                          batch_size = 5, \
                                          shuffle = False)

### Model

Fully connected and hidden layer size can be chosen.

In [ ]:
# Define AutoEncoder Model for MNIST
class MNIST_Autoencoder(nn.Module):

    def __init__(self, hidden_layer = 3):
        
        # Init from nn.Module
        super().__init__()
        
        # Encoder part will be a simple FC + ReLU.
        self.encoder = nn.Sequential(nn.Linear(28*28, hidden_layer), nn.ReLU(True))
        
        # Decoder part will be a simple FC + Tanh
        self.decoder = nn.Sequential(nn.Linear(hidden_layer, 28*28), nn.Tanh())
        

    def forward(self,x):
        
        # Forward is encoder into decoder
        x = self.encoder(x)
        x = self.decoder(x)
        return x

### Model (hidden_size = 5)

In [ ]:
# Initialize MNIST Autoencoder
torch.manual_seed(0)
model = MNIST_Autoencoder(hidden_layer = 5).to(device)

In [ ]:
# Defining Parameters
# - MSE Loss, which will be our reconstruction loss for now
# - Adam as optimizer
# - 25 Epochs
# - 128 as batch size
num_epochs = 25
batch_size = 128
distance = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), weight_decay = 1e-5)

In [ ]:
outputs_list = []
loss_list = []
for epoch in range(num_epochs):
    for data in train_loader:
        
        # Flatten image and send data to device
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).to(device)
        
        # Forward pass
        output = model(img)
        loss = distance(output, img)
        
        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # Display
    print('epoch {}/{}, loss {:.4f}'.format(epoch + 1, num_epochs, loss.item()))
    outputs_list.append((epoch, img, output),)
    loss_list.append(loss.item())

In [ ]:
plt.figure()
plt.plot(loss_list)
plt.show()

### Model (hidden_size = 25)

In [ ]:
# Initialize MNIST Autoencoder
torch.manual_seed(0)
model2 = MNIST_Autoencoder(hidden_layer = 25).to(device)

In [ ]:
# Defining Parameters
# - MSE Loss, which will be our reconstruction loss for now
# - Adam as optimizer
# - 25 Epochs
# - 128 as batch size
num_epochs = 25
batch_size = 128
distance = nn.MSELoss()
optimizer = torch.optim.Adam(model2.parameters(), weight_decay = 1e-5)

In [ ]:
outputs_list2 = []
loss_list2 = []
for epoch in range(num_epochs):
    for data in train_loader:
        
        # Flatten image and send data to device
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).to(device)
        
        # Forward pass
        output = model2(img)
        loss = distance(output, img)
        
        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # Display
    print('epoch {}/{}, loss {:.4f}'.format(epoch + 1, num_epochs, loss.item()))
    outputs_list2.append((epoch, img, output),)
    loss_list2.append(loss.item())

In [ ]:
plt.figure()
plt.plot(loss_list2)
plt.show()

### Model (hidden_size = 125)

In [ ]:
# Initialize MNIST Autoencoder
torch.manual_seed(0)
model3 = MNIST_Autoencoder(hidden_layer = 125).to(device)

In [ ]:
# Defining Parameters
# - MSE Loss, which will be our reconstruction loss for now
# - Adam as optimizer
# - 25 Epochs
# - 128 as batch size
num_epochs = 25
batch_size = 128
distance = nn.MSELoss()
optimizer = torch.optim.Adam(model3.parameters(), weight_decay = 1e-5)

In [ ]:
outputs_list3 = []
loss_list3 = []
for epoch in range(num_epochs):
    for data in train_loader:
        
        # Flatten image and send data to device
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).to(device)
        
        # Forward pass
        output = model3(img)
        loss = distance(output, img)
        
        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # Display
    print('epoch {}/{}, loss {:.4f}'.format(epoch + 1, num_epochs, loss.item()))
    outputs_list3.append((epoch, img, output),)
    loss_list3.append(loss.item())

In [ ]:
plt.figure()
plt.plot(loss_list3)
plt.show()

### Model (hidden_size = 784)

In [ ]:
# Initialize MNIST Autoencoder
torch.manual_seed(0)
model4 = MNIST_Autoencoder(hidden_layer = 784).to(device)

In [ ]:
# Defining Parameters
# - MSE Loss, which will be our reconstruction loss for now
# - Adam as optimizer
# - 25 Epochs
# - 128 as batch size
num_epochs = 25
batch_size = 128
distance = nn.MSELoss()
optimizer = torch.optim.Adam(model4.parameters(), weight_decay = 1e-5)

In [ ]:
outputs_list4 = []
loss_list4 = []
for epoch in range(num_epochs):
    for data in train_loader:
        
        # Flatten image and send data to device
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).to(device)
        
        # Forward pass
        output = model4(img)
        loss = distance(output, img)
        
        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # Display
    print('epoch {}/{}, loss {:.4f}'.format(epoch + 1, num_epochs, loss.item()))
    outputs_list4.append((epoch, img, output),)
    loss_list4.append(loss.item())

In [ ]:
plt.figure()
plt.plot(loss_list4)
plt.show()

### Visualization

In [ ]:
for data in test_loader:
    break
img, _ = data
img = img.view(img.size(0), -1)
img = Variable(img).to(device)
out1 = model(img).cpu().detach().numpy().reshape(5, 28, 28)
out2 = model2(img).cpu().detach().numpy().reshape(5, 28, 28)
out3 = model3(img).cpu().detach().numpy().reshape(5, 28, 28)
out4 = model4(img).cpu().detach().numpy().reshape(5, 28, 28)
img = img.cpu().detach().numpy().reshape(5, 28, 28)

In [ ]:
plt.figure(figsize = (20, 14))
n = 3
for i in range(n):
    plt.subplot(n, 5, 5*i + 1)
    plt.imshow(img[i])
    plt.subplot(n, 5, 5*i + 2)
    plt.imshow(out1[i])
    plt.subplot(n, 5, 5*i + 3)
    plt.imshow(out2[i])
    plt.subplot(n, 5, 5*i + 4)
    plt.imshow(out3[i])
    plt.subplot(n, 5, 5*i + 5)
    plt.imshow(out4[i])

In [ ]:
plt.figure(figsize = (10, 7))
plt.plot(loss_list, label = "hidden_layer = 5")
plt.plot(loss_list2, 'r', label = "hidden_layer = 25")
plt.plot(loss_list3, 'k', label = "hidden_layer = 125")
plt.plot(loss_list4, 'g', label = "hidden_layer = 784")
plt.legend(loc = 'best')
plt.show()